### Radiative-convective equilibrium as an idealized problem for climate

### Background on konrad 

`konrad` is a modern implementation of a single-column radiative-convective equilibrium model. In other sections of the course you'll be using very small models for which it will be straightforward enough to read the entire implementation. `konrad` is more involved so the focus will be on framing and answering questions and interpreting the results. 

The model's technical documention -- lists of all the components and functions -- is available on [ReadTheDocs](https://konrad.readthedocs.io). The konrad authors' ["How to konrad"](https://atmtools.github.io/konrad) Jupyter book is a tutorial in using the model. All the questions we asked below are found in the tutorial but you should arrive at your own answers. 

### The assignment

In the next two labs we will use konrad to explore questions of climate, including characterizing forcing, feedbacks, and climate sensitivty. Much of the material is covered in  Kluft et al. 2019, Re-examining the first climate models: climate sensitivity of a modern radiative–convective equilibrium model, doi:[10.1175/JCLI-D-18-0774.1](https://doi.org/10.1175/JCLI-D-18-0774.1).

#### Preliminaries (do in class) 

The konrad developers have provided ...

In [ ]:
# conda install -y colorcet seaborn 

In [ ]:
# Import needed Python modules 
import matplotlib.pyplot as plt
# import seaborn as sns
# import colorcet as cc

import numpy as np

import konrad

In [ ]:
# Choose pressure levels according to Equation 1 in Kluft et al. 2019 
# https://konrad.readthedocs.io/_autosummary/konrad.utils.get_quadratic_pgrid.html#konrad.utils.get_quadratic_pgrid
phlev = konrad.utils.get_quadratic_pgrid(1000e2, 10, 128) 

# Initial temperature profiles and composition 
atmosphere = konrad.atmosphere.Atmosphere(phlev)
atmosphere.atmosphere_variables

konrad defines an RCE calculation as a [set of choices](https://konrad.readthedocs.io/_autosummary/konrad.core.RCE.__init__.html). Many of the most important choices (radiation, ozone, humidity, surface, cloud, convection/vertical mixing, lapse rate, and large-scale upwelling) are specified as functions rather than discrete values. Start with the default configuation: 

In [ ]:
atmosphere["CO2"][:] = 280E-6
initial_conds = atmosphere.copy()
baseline = konrad.RCE(atmosphere, timestep='24h', max_duration='150d')

Many of the sub-models have variables associated with them: 

In [ ]:
baseline.radiation.data_vars 

Two interleaved grids are used in the vertical: state variables (temperature, concentration, humidity) are on full levels (`plev`), while fluxes (especially radiation variables) are on half-levels (`phlev`). The surface is at `phlev[0]` and there's one more half-level than full level. 

Variables are undefined until the model is run, normally to convergence, which takes about 10 seconds on the LEAPangeo hub

In [ ]:
baseline.run() 

Variables will now have valid values. The first axis is time but, by default, only the values at equilibrium are available: 

In [ ]:
baseline.radiation["lw_flxu"].shape

Note that the `atmosphere` variable itself also has the new values 

In [ ]:
print(np.max(atmosphere['T'] - baseline.atmosphere['T']))

We can, for example, plot the arbitrary initial and final equilibrium profiles of temperature. We'll plot these as a function of pressure, which varies over three orders of magnitude (so we use a log scale) and decreases as altitide increases (so we'll plot an inverted y-axis). 

In [ ]:
# sns.set_context("paper")

fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=[12, 6], sharey=True)
ax0.semilogy(initial_conds['T'][0,:], initial_conds['plev'])
ax0.semilogy(atmosphere   ['T'][0,:], initial_conds['plev'])
ax0.set_ylabel("$p$ / Pa")
ax0.set_xlabel("$T$ / K")
ax0.invert_yaxis()

ax1.semilogy(initial_conds['H2O'][0,:], initial_conds['plev'])
ax1.semilogy(atmosphere   ['H2O'][0,:], initial_conds['plev'])
ax1.set_xlabel("water vapor / kg/kg")

# sns.despine() 

What does the energy budget look like in the baseline RCE state? We can plot the four components of the radiation budget: up- and down-going radiation for solar (shortwave or sw) and terrestrial (longwave or lw) radiation. The radiation model also tells us the net heating rate. 

In [ ]:
# If we were doing this for publication we'd assign each variable a distinct color 

# sns.set_context("paper")

fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=[12, 6], sharey=True)
for v in ['lw_flxd', 'lw_flxu', 'sw_flxd', 'sw_flxu']:
    ax0.semilogy(baseline.radiation[v][0,:], initial_conds['phlev'], label=v)
ax0.set_ylabel("$p$ / hPa")
ax0.set_xlabel("$F$ / Wm$^{-2}$")
ax0.legend(frameon = False) 
ax0.invert_yaxis()


ax1.semilogy(baseline.radiation['net_htngrt'][0,:],       initial_conds['plev'])
ax1.set_ylabel("$p$ / hPa")
ax1.set_xlabel("$\mathcal{H}$ / K d$^{-1}$")

# sns.despine() 

Compare the down-going and up-going amounts of solar and terrestrial radiation. Do you understand why they have the relative values they do?  

At pressures greater than 1000 Pa, or 100 hPa, i.e. in the lowest 90% of mass in the atmopheres, there's a non-zero radiative cooling rate of roughly 1 $K/d$. That's the cooling that's making the atmosphere unstable and leading to convection. 

The upper part of the atmosphere -- the stratosphere and above -- is in radiative equilibrium. 

What is the total top-of-atmosphere energy imbalance? You'll see it's far from 0, meaning that the tropical region represented by RCE absorbs more sunlight than it emits, on average. On earth this energy is exported from the tropics towards the poles by the atmospheric and oceanic circulations. 

#### Question 1 
What is the temperature struture of the (tropical) atmosphere under present-day conditions (that's the figure above), and how does this depend on surface temperature and humidity ?

**Surface temperature**: By [default](https://konrad.readthedocs.io/_autosummary/konrad.core.RCE.html) konrad uses a [fixed surface temperature of 288 K](https://konrad.readthedocs.io/_autosummary/konrad.surface.FixedTemperature.html#konrad.surface.FixedTemperature). This can be changed by specifying a different surface temperaure when setting up the RCE calculation:  

In [ ]:
pert_Ts_286 = konrad.RCE(atmosphere, timestep='24h', max_duration='150d', 
                         surface=konrad.surface.FixedTemperature(temperature=286))
pert_Ts_286.run()

In [ ]:
# Show how to change the humidity - actually what sets the default/initial profile?

- Plot the vertical structure of temperature for several values of surface temperature and humidity. It'll be more interesting to plot the difference from the surface temperature so all the figures are on the same scale
- The "lapse rate" is the change in temperature with height or pressure. Does this depend on the initial surface temperature? On the humidity? 

#### Question 2
What is the instantaneous radiative forcing due to doubling CO$_2$ concentrations from pre-industrial values, and how does this depend on humidity and temperature? 

First it'll be useful to write a function that tells us the net downward radiation (down minus up, both shortwave and longwave) 

In [ ]:
def compute_N(konrad_instance): 
    """
    Compute net downward radiation from down- and up-going longwave and shortwave radiation 
    """
    return(konrad_instance.radiation["lw_flxd"] - konrad_instance.radiation["lw_flxu"] + 
           konrad_instance.radiation["sw_flxd"] - konrad_instance.radiation["sw_flxu"])

**Computing radiation fields**: Each component of konrad can be run on its own, so we can compute the instantaneous radiative forcing $F_i$ as the change in net radiation cased by a change in CO$_2$ concentration, i.e. 

In [ ]:
# Re-equilibrate the temperature and humidity fields to baseline conditions
baseline = konrad.RCE(atmosphere, timestep='24h', max_duration='150d')
baseline.run() 

# What is the profile of net radiation under baseline conditions? 
N_prof = compute_N(baseline)

# Calculate OLR at perturbed atmospheric state.
atmos_2xCO2 = atmosphere.copy() 
# double the CO2 concentration
atmos_2xCO2["CO2"][:] *= 2

# Compute new radiation fields with changing temperature or humidity 
baseline.radiation.update_heatingrates(atmos_2xCO2)

# The change in the net radiation field as a function of pressure 
inst_rad_forcing = (compute_N(baseline) - N_prof)[0,:]

- Plot the profiles of $F_i$ for the baseline case
- Plot the dependence of $F_i$ at the top of the atmosphere on surface temperature and/or humidity for several values of each variable

#### Question 3 
What is the effective radiative forcing due to doubling CO2 concentrations from pre-industrial values, and what is the relationship between stratospheric cooling and the radiative adjustment? 

Effective radiative forcing $F$ is the change in N (normally at the top of the atmosphere) allowing all aspects of the system _except the surface temperature_ to respond to the forcing agent. Recall, though, that konrad specifices surface temperature as fixed, so $F$ can be computed by changing CO$_2$ concentrations, running the model to equilirium with the same surface temperture, and comparing the net radiation to the unperturbed state. 

- Compute the effective radiative forcing for CO$_2$ concentrations doubled from pre-industrial concentrations (e.g. from 280 -> 560 ppmv). Compare the profile of the effective radiative forcing to the profile of instantaneous radiative forcing in question 3. 
- What fraction of $F$ is the adjustment by stratospheric cooling? 
- Plot the stratospheric adjustment - the change in stratospheric temperature profiles - for 2$\times$CO$_2$ and 0.5$\times$CO$_2$.
- Plot the top-of-atmosphere ERF for several values of surface temperature and/or humidity. Is the dependence the same as for $F_i$? 

#### Question 4

What is the climate sensitivity in radiative-convective equlibrium, and how does this depend on humidity and/or initial surface temperature? 

**Climate sensitivity, method 1**

Estimate climate sensitivty by computing the climate feedback and combining this with estimates of forcing by doubled CO2 contentrations. 

- Establish one or more baseline simulations of RCE, similar to those used in question 1, but varying humidity and/or surface temperature.  
- Change the surface temperature, compute the change in top-of-atmosphere net radiation $N$, and the climate feedback $\lambda = \frac{d N}{d T_s}$. Does $\lambda$ depend on humidity or surface temperature? 
- Estimate climate sensitivity $S$ from the effective radiative forcing (Question 3) and the climate feedback $S = \frac{F_{2\times CO_2}}{\lambda}$

**Climate sensitivity, method 2**

- From the baseline simulations, increasing $CO_2$ concentrations by a factor of 2 and run to equilibrium. (Need to use slab ocean) 
- How do estimates of $S$ compare to those estimated more indirectly? 
- How do estimates of $\lambda$ inferred from $S$ and $F$ compare to those estimated from changing surface temperature? 